# 📊 BÁO CÁO TỔNG HỢP BASE.VN - NOTEBOOK

Notebook hoàn chỉnh với 5 module Base.vn chính.

## 1. 📦 Import & Config

In [ ]:
import sys, os, warnings, pytz
import pandas as pd
from datetime import datetime
sys.path.append('.')

# Import Modules
try:
    from app_v2_all import get_employee_info_from_api, create_email_html, format_ai_content_to_html, generate_ai_insight, hcm_tz, API_KEYS, EMAIL_GUI, MAT_KHAU
    from checkin_timeoff import get_checkin_data, CheckinLoader, DetailedAttendanceAnalyzer
    from wework import get_wework_data, WeWorkAPIClient
    from goal import get_goal_data, GoalAPIClient
    from inside import get_inside_data
    from workflow import get_workflow_data
    print('✅ Imported modules from local scripts successfully')
except ImportError as e:
    print(f'❌ Import Error: {e}')
    print('⚠️ Please ensure python files (app_v2_all.py, etc.) are in the same directory')

warnings.filterwarnings('ignore')


## 2. 🔧 Helper & Shared Functions

---
# 🏢 BASE.VN MODULES
---

## 📋 WeWork
**Mô tả**: Quản lý công việc và task

**Dữ liệu**:
- Danh sách task của nhân viên
- Trạng thái: Done / Doing / Pending
- Tỷ lệ hoàn thành, đúng hạn
- Task quá hạn, sắp đến hạn

## 🎯 Goal
**Mô tả**: Quản lý OKR (Objectives & Key Results)

**Dữ liệu**:
- Goals và Key Results theo cycle (quý)
- Tiến độ hiện tại, biến động tuần qua
- Hành vi check-in (số lần, tần suất)

## ⏰ Checkin + Timeoff
**Mô tả**: Chấm công và nghỉ phép

**Dữ liệu**:
- Số ngày công trong tháng
- Số lần đi trễ / về sớm
- Giờ check-in trung bình
- Phân tích thói quen (Early Bird / Punctual / Late)

## 📢 Inside
**Mô tả**: Mạng xã hội nội bộ, tin tức công ty

**Dữ liệu**:
- Số bài viết đã đăng
- Lượt reactions nhận/cho
- Lượt views nhận/cho
- Phân loại vai trò (Influencer / Observer / Ghost)

## 🔄 Workflow
**Mô tả**: Quản lý quy trình công việc

**Dữ liệu**:
- Danh sách jobs của nhân viên
- Trạng thái: Done / Doing
- Jobs quá hạn, sắp đến hạn

---
# 🚀 MAIN & TEST
---

## 🤖 AI & Email Functions
**Mô tả**: Tích hợp AI (Ollama) để phân tích dữ liệu và tạo nội dung email HTML.

- **`generate_ai_insight`**: Gửi dữ liệu JSON context của từng module (WeWork, Goal, Workflow) sang Ollama (Model: `gemini-3-flash-preview`) để tạo nhận xét (insight) và khuyến nghị hành động (recommendations).
- **`format_ai_content_to_html`**: Chuyển đổi phản hồi của AI (Markdown/Bullet points) thành HTML thẻ `<div>` đẹp mắt để nhúng vào email.
- **`create_email_html`**: Hàm tổng hợp (Master function) tạo toàn bộ nội dung email báo cáo, kết hợp dữ liệu cứng (stats) và dữ liệu mềm (AI insights) vào một template HTML clean, hiện đại.
- **`send_email`**: Gửi email thông qua SMTP Gmail.

### 📝 Note on AI Processing
- **Model**: Gemini-3-flash-preview (via Ollama)
- **Privacy**: Data is processed locally/via private API.
- **Fallback**: Automatic key rotation enabled for API stability.


## 🧪 Test Run

In [ ]:

# ==============================================================================
# 🚀 MAIN EXECUTION
# ==============================================================================

def run_report_demo(target_name="Hoang Tran"):
    print("="*80)
    print(f"📧 DEMO: BÁO CÁO TỔNG HỢP CHO: {target_name.upper()}")
    print("="*80)
    
    # 1. Identify User
    user_info = get_employee_info_from_api(target_name)
    if not user_info:
        print(f"❌ Không tìm thấy nhân viên: {target_name}")
        return

    emp_name = user_info['name']
    emp_username = user_info.get('username', '')
    emp_email = user_info.get('email', '')
    join_date = user_info.get('since', '') # NEW: Get join date
    
    print(f"✅ Found: {emp_name} ({emp_username}) - Join Date: {join_date}")

    # 2. Fetch Data
    print("\n🔄 Đang tải dữ liệu...")
    
    # Server Data (Logic App)
    try:
        import app_v2_logic
        # Reload to ensure latest changes if editing interactively
        import importlib
        importlib.reload(app_v2_logic)
        server_data = app_v2_logic.get_review_user_work_plus_data(emp_name)
        print("  - Server/Logic data: OK")
    except Exception as e:
        print(f"  - Server/Logic error: {e}")
        server_data = None
        
    # Standard Modules
    # NEW: Pass join_date and date params to checkin
    today = datetime.now()
    checkin = get_checkin_data(emp_name, today.year, today.month, join_date=join_date)
    
    wework = get_wework_data(emp_username)
    goal = get_goal_data(emp_name)
    inside = get_inside_data(emp_name)
    workflow = get_workflow_data(emp_name)
    
    print("  - Base Modules data: OK")

    # --- NEW: Recent Data Filter Logic ---
    # Calculate one month ago
    from datetime import timedelta
    one_month_ago = today - timedelta(days=30)
    one_month_ago_ts = one_month_ago.timestamp()

    # Kiểm tra dữ liệu gần đây
    has_recent_wework = False
    if wework and wework.get('recent_tasks'):
        for task in wework['recent_tasks']:
            since_ts = task.get('since', 0)
            if since_ts and float(since_ts) >= one_month_ago_ts:
                has_recent_wework = True
                break

    has_recent_workflow = False
    if workflow and workflow.get('latest_jobs'):
        for job in workflow['latest_jobs']:
            date_str = job.get('date', '')
            if date_str and date_str != 'N/A':
                try:
                    job_date = datetime.strptime(date_str, '%d/%m/%Y %H:%M:%S')
                    if job_date >= one_month_ago:
                        has_recent_workflow = True
                        break
                except:
                    pass

    has_recent_goal = bool(goal and goal.get('weekly'))
    active_sections_count = sum([has_recent_wework, has_recent_workflow, has_recent_goal])

    if active_sections_count == 0:
        print(f"⚠️ {emp_name}: Không có dữ liệu gần đây. Vẫn gửi email cảnh báo.")
        wework = {
            'summary': {'total_tasks': 0},
            'is_warning_only': True
        }
        goal = None
        workflow = None
    else:
        if not has_recent_wework: wework = None
        if not has_recent_workflow: workflow = None
        if not has_recent_goal: goal = None
    # -------------------------------------

    # 3. Generate HTML
    print("\n🎨 Đang tạo HTML báo cáo (kèm AI Insight)...")
    html_content = create_email_html(emp_name, checkin, wework, goal, inside, workflow, server_data)
    
    # 4. Preview / Send
    preview_file = "notebook_report_preview.html"
    with open(preview_file, "w", encoding="utf-8") as f:
        f.write(html_content)
    print(f"✅ Đã tạo file xem trước: {preview_file}")
    
    # Uncomment to send
    # send_email(emp_email, f"BÁO CÁO TỔNG HỢP - {emp_name}", html_content)

# --- RUN ---
run_report_demo("Hoang Tran")
